In [ ]:
# tutorial on pytorch and transformers.
# https://towardsdatascience.com/how-to-train-bert-aaad00533168

In [ ]:
# install packages and modules.
# ! pip install tokenizers===0.9.3
! pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# imports
from transformers import BertTokenizer, BertForPreTraining, AdamW
import torch
# from google.colab import drive
# 
# drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# load data
text = ""
with open("./clean.txt", "r") as data_obj:
  text = data_obj.read().split('\n')

text[ : 3]

FileNotFoundError: ignored

In [ ]:
# define objects
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForPreTraining.from_pretrained('bert-base-uncased')


In [ ]:
# next sentence prediction
import random

# 
bag = [item for sentence in text for item in sentence.split('.') if item != '']
bag_size = len(bag)

# create 50/50 random and non-random nsp sentences
sentence_a = []
sentence_b = []
label = []

for paragraph in text:
  sentences = [sentence for sentence in paragraph.split('.') if sentence != '']
  num_sentences = len(sentences)
  
  if num_sentences > 1:
    start = random.randint(0, num_sentences - 2 )
    # 50/50 whether is IsNextSentence or NotNextSentence
    if random.random() >= .5:
      # this is IsNextSentence
      sentence_a.append( sentences[start] )
      sentence_b.append( sentences[start+1] )
      label.append(0)
    else:
      index = random.randint(0, bag_size-1)
      sentence_a.append( sentences[start] )
      sentence_b.append( bag[index] )
      label.append(1)

# len(sentence_a), len(sentence_b)

inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt', max_length=512, truncation=True, padding='max_length')
# ['input_ids', 'token_type_ids', 'attention_mask']

# add a label to our input, convert labels into tensor and transpose it
inputs['next_sentence_label'] = torch.LongTensor([label]).T

# clone input_ids to create masked language model label
inputs['label'] = inputs.input_ids.detach().clone()

# inputs.keys()

# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)

# create mask array
# by adding the additional logic when creating mask_arr — we are ensuring 
# that we don’t mask any special tokens — such as CLS (101), SEP (102), and PAD (0) tokens.
mask_arr = (rand < 0.15) * (inputs.input_ids != 101) * (inputs.input_ids != 102) * (inputs.input_ids != 0)

selection = []
for i in range(inputs.input_ids.shape[0]):
    selection.append( torch.flatten(mask_arr[i].nonzero()).tolist() )

for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 103

# Then apply these indices to each row in input_ids, assigning
# each value at these indices a value of 103.

In [ ]:
# create a pytorch dataset from our data
class OutDataset(torch.utils.data.Dataset):
  def __init__( self, encodings ):
    self.encodings = encodings

  def __getitem__(self, idx):
    return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  def __len__(self):
    return len(self.encodings.input_ids)

# create dataset object
dataset = OutDataset(inputs)

# define and initialise the data loader
# The dataloader expects the __len__ method for checking the 
# total number of samples within our dataset, 
# and the __getitem__ method for extracting samples.
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True )

In [ ]:
# set up for training.
from tqdm import tqdm # use for loading.

epochs = 2
device = torch.device("cuda") if torch.cuda.is_available() else torch.device('cpu')
model.to(device)
model.train()
optim = AdamW(model.parameters(), lr=5e-5)
for epoch in range( epochs):
  # setup loop with TQDM and dataloader
  loop = tqdm(loader, leave=True)
  for batch in loop:
    # initialize calculated gradients (from prev step)
    optim.zero_grad()
    # pull all tensor batches required for training
    input_ids = batch['input_ids'].to(device)
    token_type_ids = batch['token_type_ids'].to(device)
    attention_mask = batch['attention_mask'].to(device)
    next_sentence_label = batch['next_sentence_label'].to(device)
    labels = batch['label'].to(device)
    # process

    outputs = model(input_ids, attention_mask=attention_mask,
                        token_type_ids=token_type_ids,
                        next_sentence_label=next_sentence_label,
                        labels=labels)
    
    # extract loss
    loss = outputs.loss
    # calculate loss for every parameter that needs grad update
    loss.backward()
    # update parameters
    optim.step()
    # print relevant info to progress bar
    loop.set_description(f'Epoch {epoch}')
    loop.set_postfix(loss=loss.item())